In [4]:
import requests
from bs4 import BeautifulSoup
import time,re
import random , logstash , logging , pymongo



In [ ]:


host = '192.168.114.130'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Mobile01"]


In [2]:
s = "我們都很好Pleas ======="



In [5]:
re.sub(r"\W+","",s)


'我們都很好Pleas'

'我們都很好Pleas ======='

In [ ]:
# pipeline 
logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))




In [ ]:
url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&p=1"

def get_url(n=2,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&p=" + str(t)
        result.append(url)
    return result
    

In [ ]:
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

class Connect404Except(Exception):
    pass


def get_connect(url):
    i = 0
    
    while True :
        
        try :
            resp = requests.get(url,headers=header,proxies=proxylist[i])          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,4))
           
            
        except requests.exceptions.ConnectionError :             
            #logger.warning("Proxy Error",extra={"proxy":proxylist[i]})
            
            i = (i + 1) % 6
            continue

            
            
        if resp : 
            if resp.status_code == 200 :
                code = "HTTP response code = " + str(resp.status_code)
                #logger.info(code,extra={"url" : url, "time_use" : resp.elapsed.total_seconds()})
                #print(proxylist[i])
                break         
        if resp.status_code == 404 :
            #logger.warning("This page is gone nowhere !",extra={"url":url})
                
            break
        
        
    return resp
    

In [ ]:
def parse(urls):
    result = []
    
    connect = get_connect(urls)
    try :
        if connect.status_code == 404:
            raise Connect404Except
            
        else :
            soup = BeautifulSoup(connect.text)
            
    except Connect404Except : 
        logger.error("index error",extra={"index":urls})
        pass
    
    if soup :
        print("進入",urls)
        table = soup.find_all("tr")
        # 取得列表內容
        for t in table:
            title = t.find("p", class_="title").text
            href = "https://www.mobile01.com/" + t.find("p", class_="title").find("a")["href"]
            sp = t.find("p", class_="info").text.split("-")[1].replace(" ","")
            date = t.find("p", class_="info").text.split(":")[2].split(" ")[1]
            
            # 進入第二層 遊記 去除風景
            if "餐飲" in sp:
                print("現在處理:",href)
                conn = get_connect(href)
                try :
                    if conn.status_code == 404:
                        raise Connect404Except

                    else :
                        art = BeautifulSoup(conn.text)

                except Connect404Except : 
                    logger.error("page error",extra={"index":href})
                    continue
                content = art.find("div", class_="single-post-content").text.replace("\r","").replace("\n","").replace(" ","")
                img_url = []
                imgs = art.find("div", class_="single-post-content").find_all("img")
                for img in imgs:
                    img_url.append(img["data-src"])
                author = soup.find("div", class_="panel note sidebar-authur").find("a").text
                article_hit = soup.find_all("div",class_="panel note")[1].find_all("li")[0].text.split(":")[-1]
                rate = soup.find_all("div",class_="panel note")[1].find_all("li")[1].text.split(":")[-1]
                res_info = get_res_info(art)
                # 進入第三層 評論
                comment_url = "https://www.mobile01.com/" + soup.find("div", class_="btns wide").find("a")["href"]
                new_conn = BeautifulSoup(get_connect(comment_url).text)
                comment = get_comment(new_conn)
                if art.find("div", class_="pagination").find("a").text == "下一頁"
                
                    
                        
        
    



In [ ]:
table = soup.find_all("tr")
table[0]

In [ ]:
table[0].find("p", class_="info").text.split("-")[1].replace(" ","")

In [ ]:
# 取得列表內容
for t in table:
    title = t.find("p", class_="title").text
    href = "https://www.mobile01.com/" + t.find("p", class_="title").find("a")["href"]
    sp = t.find("p", class_="info").text.split("-")[1].replace(" ","")
    date = t.find("p", class_="info").text.split(":")[2].split(" ")[1]
    print("title : ",title)
    print("href : ",href)
    print("sp : ",sp)
    print("date : ",date)
    
        
    

In [ ]:
href = "https://www.mobile01.com/waypointdetail.php?id=32490"
soup = BeautifulSoup(get_connect(href).text)

In [ ]:
# 文章本文
content = soup.find("div", class_="single-post-content").text.replace("\r","").replace("\n","").replace(" ","")
img_url = []
imgs = soup.find("div", class_="single-post-content").find_all("img")
for img in imgs:
        img_url.append(img["data-src"])
author = soup.find("div", class_="panel note sidebar-authur").find("a").text
article_hit = soup.find_all("div",class_="panel note")[1].find_all("li")[0].text.split(":")[-1]
rate = soup.find_all("div",class_="panel note")[1].find_all("li")[1].text.split(":")[-1]
res_info = get_res_info(soup)


In [ ]:
# 餐廳資訊 評等
def get_res_info(soup):
    res_info = []
    info = soup.find(id="info_form")
    
    rate = len(info.find_all("div")[8].find_all("img"))
    place = info.find_all("div")[1].text.replace("\n","").replace(" ","").split(":")[-1]
    phone = info.find_all("div")[2].text.replace("\n","").replace(" ","").split(":")[-1]
    op_time = info.find_all("div")[3].text.replace("\n","").split("  ")[-1]
    point = info.find_all("div")[7].text.replace("\n","").replace(" ","").split(":")[-1]
    res_info.append({
        "rate" : rate,
        "place" : place,
        "phone" : phone,
        "op_time" : op_time,
        "point" : point
    })
    return res_info



In [ ]:
res_info

In [ ]:
comment_url = "https://www.mobile01.com/" + soup.find("div", class_="btns wide").find("a")["href"]

comment_soup = BeautifulSoup(get_connect(comment_url).text)


In [ ]:
get_comment(comment_soup)

In [ ]:
comment = []
for single_comment in comment_soup.find_all("article")[1:]:
    removes = single_comment.find_all("span", class_="poster")
    for remove in removes:
        remove.extract()
    removes = single_comment.find_all("blockquote")
    for remove in removes:
        remove.extract()
    author = single_comment.find("div", class_="single-post-author group").find("div", class_="fn").text
    content = single_comment.find("div", class_="single-post-content").text.replace("\n","").replace("\r","").replace(" ","")
    content = author + ":" + content
    comment.append(content)
                                                                                                     
    

In [ ]:
def get_comment(soup):
    comment = []
    for single_comment in soup.find_all("article")[1:]:
        removes = single_comment.find_all("span", class_="poster")
        for remove in removes:
            remove.extract()
        removes = single_comment.find_all("blockquote")
        for remove in removes:
            remove.extract()
        author = single_comment.find("div", class_="single-post-author group").find("div", class_="fn").text
        content = single_comment.find("div", class_="single-post-content").text.replace("\n","").replace("\r","").replace(" ","")
        content = author + ":" + content
        comment.append(content)
    
    return comment


In [ ]:
def get_comment_p2(soup):
    comment = []
    for single_comment in soup.find_all("article"):
        removes = single_comment.find_all("span", class_="poster")
        for remove in removes:
            remove.extract()
        removes = single_comment.find_all("blockquote")
        for remove in removes:
            remove.extract()
        author = single_comment.find("div", class_="single-post-author group").find("div", class_="fn").text
        content = single_comment.find("div", class_="single-post-content").text.replace("\n","").replace("\r","").replace(" ","")
        content = author + ":" + content
        comment.append(content)
    
    return comment

In [ ]:
art = BeautifulSoup(get_connect("https://www.mobile01.com/waypointtopicdetail.php?f=195&t=5694705#2").text)

In [ ]:
links = art.find_all("div",class_="pagination")[1].find_all("a")
for link in links:
    if link.find(string=re.compile("下一頁")):
        print(link)
links

In [ ]:
import re
# stackoverflow solution 
# https://stackoverflow.com/questions/31958637/beautifulsoup-search-by-text-inside-a-tag
art2 = BeautifulSoup(get_connect("https://www.mobile01.com/waypointtopicdetail.php?f=189&t=4462255").text)
links = art2.find_all("div",class_="pagination")[1].find_all("a")
for link in links :
    if link.find(string=re.compile("下一頁")):
        print("https://www.mobile01.com/"+link["href"])
    

In [ ]:
re.compile("下一頁 ›")

In [ ]:
resp = BeautifulSoup(get_connect("https://www.ptt.cc/bbs/Food/M.1523617167.A.44E.html").text)

In [ ]:
if resp.find_all("div", class_="push"):
    print("A")
  